### Buy and Hold
1. 매수 후 일정 기간 동안 유지 매도

In [ ]:
import pandas as pd

In [ ]:
## csv 폴더 안에 있는 AAPL.csv 로드 
df = pd.read_csv("../../csv/AAPL.csv")
df.head()

In [ ]:
## Date 컬럼을 인덱스로 변경 
df.set_index('Date', inplace=True)

In [ ]:
df2 = pd.read_csv("../../csv/AAPL.csv", index_col=0)
df2.head()

In [ ]:
## 데이터프레임의 정보를 확인 
df.info()

In [ ]:
# 결측치의 개수 확인 
df.isna().sum()

In [ ]:
# 데이터중 결측치가 포함된 인덱스를 확인 
flag = df.isna().any(axis=1)

In [ ]:
# 결측치가 포함된 데이터를 필터링 
df.loc[flag, ]

In [ ]:
# 결측치가 포함되지 않는 데이터를 필터링 
df.loc[~flag, ]

In [ ]:
import numpy as np

In [ ]:
# 결측치, 무한대 값들은 제외한다. 
flag2 = df.isin([np.nan, np.inf, -np.inf]).any(axis=1)

In [ ]:
df = df.loc[~flag2,]

In [ ]:
## 특정 컬럼(Adj Close)을 제외하고 나머지 컬럼들은 모두 제거 
AAPL = df[['Adj Close']]

In [ ]:
## 일별 수익율 계산하여 새로운 파생변수에 대입 
# 오늘의 수정주가 / 전날의 수정주가 -> price_AAPL 변수에 대입 
# price_AAPL를 데이터프레임 daily_rtn 컬럼에 대입
price_AAPL = AAPL['Adj Close'] / AAPL['Adj Close'].shift(1)
AAPL['daily_rtn'] = price_AAPL.fillna(1)

In [ ]:
AAPL.head()

In [ ]:
# 수익율 계산 
# 일별 수익율을 모두 누적곱 
rtn = 1

for i in range(len(AAPL)):
    # i 가 의미하는것은? 인덱스(위치)
    # print(AAPL.iloc[i,1])
    rtn *= AAPL.iloc[i, 1]
rtn

In [ ]:
AAPL.iloc[-1, 0] / AAPL.iloc[0, 0]

In [ ]:
## 일별 수익율 계산 (함수를 이용해서 )
AAPL['daily_rtn2'] = (AAPL['Adj Close'].pct_change() + 1).fillna(1)

In [ ]:
# 총 수익율 계산 
## 일별 수익율을 누적곱
AAPL['daily_rtn2'].cumprod()

In [ ]:
AAPL.index

In [ ]:
from datetime import datetime

In [ ]:
## 문자형 데이터를 시계열 데이터로 변경 
start = "201806"
print(type(start))

In [ ]:
start = datetime.strptime(start, '%Y%m')
print(start)
print(type(start))

In [ ]:
# AAPL의 index를 시계열 데이터로 변환 
datetime.strptime(AAPL.index[0], '%Y-%m-%d')

In [ ]:
## 반복문을 이용해서 시계열로 변경 
index_to_date = []

for i in range(len(AAPL)):
    date_data = datetime.strptime(AAPL.index[i], '%Y-%m-%d')
    index_to_date.append(date_data)

index_to_date

In [ ]:
list(
    map(
        lambda x : datetime.strptime(x, '%Y-%m-%d'), 
        AAPL.index
    )
)

In [ ]:
## pandas 내장 함수를 이용해서 시계열 데이터로 변환
## to_datetime({1차원 데이터}, format = {데이터들의 형태})
AAPL.index = pd.to_datetime(AAPL.index, format="%Y-%m-%d")

In [ ]:
## 매수의 일, 매도의 일 기준으로 데이터를 필터링 
start = "2007년01월01일"
end = "2010년12월31일"

start = datetime.strptime(start, '%Y년%m월%d일')
end = datetime.strptime(end, '%Y년%m월%d일')

In [ ]:
price_df = AAPL.loc[ start : end,  ['Adj Close']]

In [ ]:
## 일별 수익율 컬럼을 생성 
daily_rtn = (price_df['Adj Close'] / price_df['Adj Close'].shift()).fillna(1)

In [ ]:
price_df['daily_rtn'] = daily_rtn

In [ ]:
price_df['daily_rtn2'] = (price_df['Adj Close'].pct_change() + 1).fillna(1)

In [ ]:
## 누적 수익율 계산
rtn = 1

for i in range(len(price_df)):
    rtn *= price_df.iloc[i, 1]

rtn

In [ ]:
price_df['rtn'] = price_df['daily_rtn2'].cumprod()

In [ ]:
price_df.tail()

In [ ]:
## Buy and Hold 함수화 
## 매개변수는 데이터프레임, 기준이 되는 컬럼, 시작시간, 종료시간
## 기준이 되는 컬럼은 기본값이 Adj Close
## 시작시간은 2000-01-01을 기본값
## 종료시간은 오늘 날짜가 기본값

def buyandhold(
        _df, 
        _col = 'Adj Close', 
        _start = '2010-01-01', 
        _end = datetime.now()):
    try:
        # _start 는 시계열 데이터로 변경 
        start = datetime.strptime(_start, '%Y-%m-%d')
        # _end는 문자형 데이터라면 시계열로 변경 
        # 시계열 데이터라면 end 변수에 _end를 그대로 대입
        if type(_end) == 'str':
            end = datetime.strptime(_end, '%Y-%m-%d')
        else:
            end = _end
    except:
        return "인자값의 타입이 잘못되었습니다.(예 : YYYY-mm-dd)"
    # _df의 복사본을 생성
    df = _df.copy()
    # 컬럼에 Date 포함되어있는가? 
    if 'Date' in df.columns:
        # Date 컬럼을 인덱스로 변경 
        df.set_index('Date', inplace=True)
    # index를 시계열 데이터로 변경 
    df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
    # 시간시간과 종료시간을 기준으로 필터
    # 특정 컬럼의 데이터만 필터
    result = df.loc[ start : end , [_col] ]
    # 일별 수익율 컬럼을 생성 
    result['daily_rtn'] = (result[_col].pct_change() + 1).fillna(1)
    # 누적수익율을 계산하여 새로운 컬럼에 대입 
    result['acc_rtn'] = result['daily_rtn'].cumprod()
    acc_rtn = result.iloc[-1, 2]
    return result, acc_rtn



In [ ]:
df = pd.read_csv('../../csv/AMZN.csv')
df2 = pd.read_csv("../../csv/GM.csv", index_col='Date')

In [ ]:
df.head(1)

In [ ]:
df2.head(1)